In [1]:
import os
from urllib import request
from zipfile import ZipFile

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

torch.manual_seed(360);

# Load data

In [11]:
def download_and_extract_data(dataset_dir=""):
    data_zip = os.path.join(dataset_dir, "hymenoptera_data.zip")
    data_path = os.path.join(dataset_dir, "hymenoptera_data")
    url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

    if not os.path.exists(data_path):
        if not os.path.exists(data_zip):
            print("Starting to download data...")
            data = request.urlopen(url, timeout=300).read()
            with open(data_zip, "wb") as f:
                f.write(data)

        print("Starting to extract data...")
        with ZipFile(data_zip, "r") as zip_f:
            zip_f.extractall(dataset_dir)

    print("Data has been downloaded and extracted to {}.".format(dataset_dir))


download_and_extract_data()

Starting to extract data...
Data has been downloaded and extracted to .


In [13]:
data_dir = "hymenoptera_data"
train_transforms = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)
val_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

train_ds = datasets.ImageFolder(os.path.join(data_dir, "train"), train_transforms)
val_ds = datasets.ImageFolder(os.path.join(data_dir, "val"), val_transforms)

# Pretrained model
- fc layer of pretrained model is replaced by new one, which has desired new shape

In [14]:
class PretrainedModel(nn.Module):
    def __init__(self, output_features):
        super().__init__()
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, output_features)
        self.model = model

    def forward(self, x):
        return self.model(x)

In [18]:
models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/hyeonwoo.yoo/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 28.9MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Problem

We are going to train a neutral network to classify ants and bees. The dataset consist of 120 training images and 75 validiation images for each class. First we create the training and validiation datasets:

In [15]:
from skorch.callbacks import LRScheduler

lrscheduler = LRScheduler(policy="StepLR", step_size=7, gamma=0.1)

In [16]:
from skorch.callbacks import Checkpoint

checkpoint = Checkpoint(f_params="best_model.pt", monitor="valid_acc_best")

In [17]:
from skorch.callbacks import Freezer

freezer = Freezer(lambda x: not x.startswith("model.fc"))

In [21]:
net = NeuralNetClassifier(
    PretrainedModel,
    criterion=nn.CrossEntropyLoss,
    lr=0.001,
    batch_size=4,
    max_epochs=25,
    module__output_features=2,
    optimizer=optim.SGD,
    optimizer__momentum=0.9,
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
    iterator_valid__shuffle=True,
    iterator_valid__num_workers=4,
    train_split=predefined_split(val_ds),
    callbacks=[lrscheduler, checkpoint, freezer],
    device=device,  # comment to train on cpu
)

In [22]:
net.fit(train_ds, y=None);

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.5548       0.9085        0.2250     +  1.4044
      2        0.4998       0.9281        0.1606     +  1.3422
      3        0.3917       0.9085        0.2415        1.3867
      4        0.6467       0.9412        0.1689     +  1.4145
      5        0.3563       0.9412        0.1674        1.4652
      6        0.4419       0.9412        0.1822        1.3694
      7        0.4510       0.9542        0.1700     +  1.3651
      8        0.3815       0.9608        0.1490     +  1.3144
      9        0.3618       0.9542        0.1527        1.3403
     10        0.3878       0.9542        0.1513        1.2990
     11        0.2846       0.9608        0.1591        1.3399
     12        0.4086       0.9477        0.1608        1.3546
     13        0.3806       0.9412        0.1533        1.3416
     14        0.4402       0.9542        0.1547       